<a href="https://colab.research.google.com/github/brenoakihiromorimoto/portf-lio/blob/main/AMAbilidados/ama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Análise das Notas Fiscais Paulista

Por uma descrição aqui...

## Análise antes de gerar a base final

### Carga das bibliotecas e configurações de display

In [ ]:
# Carga das bibliotecas
import os

import pandas as pd

In [ ]:
# Configurações de display
pd.options.display.max_columns = None

### Carga e combinação dos dados

#### Dados da AMA

In [ ]:
# Listando todos os arquivos do diretório de dados da AMA
paths_ama = os.listdir('./dados_ama/')
paths_ama

In [ ]:
# Carga inicial apenas para verificarmos o formato de cada uma das bases
for cont in range(len(paths_ama)):
    caminho = './dados_ama/' + paths_ama[cont]
    display(pd.read_csv(caminho, sep='\t', encoding='utf-16', nrows=1))

In [ ]:
# Concatenando os dados das bases
df_ama = pd.DataFrame()

for cont in range(len(paths_ama)):
    caminho = './dados_ama/' + paths_ama[cont]
    df_ama = pd.concat([df_ama, pd.read_csv(caminho, sep='\t', encoding='utf-16')], ignore_index=True)

In [ ]:
# Observando o formato da base resultante
df_ama.shape

In [ ]:
# Verificando os índices das últimas observações 
df_ama.tail()

#### Dados da receita

1) Dados dos estabelecimentos

In [ ]:
# Listando todos os arquivos do diretório de dados dos estabelecimentos
paths_estabelecimentos = os.listdir('./dados_receita/estabelecimentos')
paths_estabelecimentos

In [ ]:
# Carga inicial apenas para confirmarmos o formato de cada uma das bases
for cont in range(len(paths_estabelecimentos)):
    caminho = './dados_receita/estabelecimentos/' + paths_estabelecimentos[cont]
    display(pd.read_csv(caminho, sep=';', encoding='cp437', nrows=1))

Observe que os dados estão sem labels para as colunas. Consultando o dicionário da base de dados dos estabelecimentos [link](https://www.gov.br/receitafederal/pt-br/assuntos/orientacao-tributaria/cadastros/consultas/arquivos/novolayoutdosdadosabertosdocnpj-dez2021.pdf), obtemos tais labels, assim como os seus significados.

In [ ]:
# As chaves serão os nomes das colunas e os valores serão os índices das colunas a serem carregadas
colunas_indices = {'cnpj_basico': 0, 'cnpj_ordem': 1, 'cnpj_div': 2,
                  'id_matriz': 3, 'nome_fantasia': 4, 'situacao_cadastral': 5,
                   'cnae_principal': 11, 'bairro': 17}

nomes = colunas_indices.keys()
selecionadas = colunas_indices.values()

In [ ]:
# Teste de velocidade para a carga da primeira base
caminho = './dados_receita/estabelecimentos/' + paths_estabelecimentos[0]
display(pd.read_csv(caminho, sep=';', names=nomes, encoding='cp437', usecols=selecionadas)[:1])

In [ ]:
# Concatenando os dados das bases
df_estabelecimentos = pd.DataFrame()

for cont in range(len(paths_estabelecimentos)):
    caminho = './dados_receita/estabelecimentos/' + paths_estabelecimentos[cont]
    df_estabelecimentos = pd.concat([df_estabelecimentos,
                                     pd.read_csv(caminho, sep=';', encoding='cp437',
                                                 on_bad_lines='skip', names=nomes, 
                                                 usecols=selecionadas)], ignore_index=True)

In [ ]:
# Observando o formato da base resultante
df_estabelecimentos.shape

In [ ]:
# Conferindo as últimas linhas da base gerada
df_estabelecimentos.tail()

2) Dados da CNAE

In [ ]:
df_cnae = pd.read_csv('./dados_receita/CNAE/F.K03200$Z.D20514.CNAECSV',
                      sep=';', encoding='cp437', names=['codigo', 'descricao'])
df_cnae.head()

### Tratamento dos dados

#### Base da AMA

In [ ]:
# Observando 5 amostras aleatórias da base
df_ama.sample(5)

In [ ]:
# Modificando o nome das variáveis
df_ama.rename(columns={'CNPJ emit.': 'cnpj', 'Emitente.': 'emitente',
                       'Data Emissão': 'data_emissao',
                       'Valor NF': 'valor_nf', 'Créditos': 'creditos',
                       'Situação do Crédito': 'situacao_credito',
                       'Emitente': 'emitente'}, inplace=True)
df_ama[:2]

In [ ]:
# Consultando algumas informações gerais da base
df_ama.info()

In [ ]:
## Modificando os tipos de algumas variáveis

# Convertendo strings em datas
df_ama['data_emissao'] = pd.to_datetime(df_ama['data_emissao'], format='%d/%m/%Y')

# Convertendo strings em decimais
df_ama['valor_nf'] = df_ama['valor_nf'].str.replace(',', '.')
df_ama['valor_nf'] = df_ama['valor_nf'].astype(float)
df_ama['creditos'] = df_ama['creditos'].str.replace(',', '.')
df_ama['creditos'] = df_ama['creditos'].astype(float)

# Verificando os novos tipos de dados
df_ama.dtypes 

In [ ]:
# Ordenando os dados pela data de emissão
df_ama = df_ama.sort_values('data_emissao', ignore_index=True)
df_ama.head()

In [ ]:
# Verificando se há linhas duplicadas
df_ama.duplicated().sum()

Para fazermos um mapeamento do CNPJ com os dados da receita federal precisaremos que o CNPJ tenha o padrão 'xx.xxx.xxx/xxxx-xx'. Sendo assim, iremos conferir se há valores de CNPJ diferentes deste formato:

In [ ]:
regex_cnpj = r'\d{2}\.\d{3}\.\d{3}/\d{4}-\d{2}'
(df_ama['cnpj'].str.match(regex_cnpj) == False).sum()

In [ ]:
# Removendo as colunas que não são de interesse 
df_ama.drop(columns=['No.', 'Data Registro'], inplace=True)
df_ama.columns

#### Dados dos Estabelecimentos

In [ ]:
# Verificando o formato da base
df_estabelecimentos.shape

In [ ]:
# Verificando se há observações repetidas
df_estabelecimentos.duplicated().sum()

Faz sentido termos tantos CNPJs (aproximadamente 1/4 da população do país) únicos? Segundo dados obtidos pelo portal [correio dos munícipios](https://www.correiodosmunicipios-al.com.br/2021/10/brasil-registra-recorde-em-abertura-de-empresas-no-2o-quadrimestre/#:~:text=No%20segundo%20quadrimestre%20de%202021,recorde%20na%20hist%C3%B3ria%20do%20Brasil.), no segundo quadrimestre de de 2021 o número de CNPJs ativos era de 18.440.986. Pegando esse valor como referência, podemos filtrar a base pelas ocorrências de CNPJs ativos na base cuja situação cadastral está ativa (situacao_cadastral = 2):

In [ ]:
df_estabelecimentos['situacao_cadastral'].value_counts()

Uma vez que os últimos dados registrados foram de maio de 2022, 21811232 parece ser um número factível.

In [ ]:
# Observando 5 amostras aleatórias da base
df_estabelecimentos.sample(5)

Note que os dados do CNPJ encontram-se nas 3 primeiras variáveis. No padrão CNPJ, o CNPJ básico é representado pelos 8 primeiros dígitos; o CNPJ ordem corresponde aos dígitos entre o oitavo e o décimo segundo; o CNPJ div é um número de dois dígitos, sendo os dois últimos dígitos do padrão completo.

Com o objetivo de concatenarmos essees padrões e definirmos o padrão completo, considere a a função compor_cnpj:

In [ ]:
def compor_cnpj(df):
    df['cnpj_basico'] = df['cnpj_basico'].apply(lambda x: f'{x:0>8}')
    df['cnpj_ordem'] = df['cnpj_ordem'].apply(lambda x: f'{x:0>4}')
    df['cnpj_div'] = df['cnpj_div'].apply(lambda x: f'{x:0>2}')
    df['cnpj'] = df['cnpj_basico'] + df['cnpj_ordem'] + df['cnpj_div']
    df.drop(columns=['cnpj_basico', 'cnpj_ordem', 'cnpj_div'], inplace=True)
    return df

In [ ]:
# Formando o CNPJ n padrao xxxxxxxxxxxxxx
df_estabelecimentos = compor_cnpj(df_estabelecimentos)

In [ ]:
df_estabelecimentos.head()

Como os CNPJs da base da AMA estão no padrão xx.xxx.xxx\xxxx-xx, para que possamos comparar estes valores eles precisam estar no mesmo padrão. Sendo assim, iremos aplicar uma formatação nos valores dos CNPJs:

In [ ]:
def ajustar_cnpj(df):
    df['cnpj'] = (df['cnpj'].str[:2] + '.' + 
                  df['cnpj'].str[2:5] + '.' + 
                  df['cnpj'].str[5:8] + '/' + 
                  df['cnpj'].str[8:12] + '-' + 
                  df['cnpj'].str[12:])
    return df

In [ ]:
# Formando o CNPJ no padrão xx.xxx.xxx\xxx-xx
df_estabelecimentos = ajustar_cnpj(df_estabelecimentos)

In [ ]:
df_estabelecimentos.head()

In [ ]:
# Conferindo os tipos de dados
df_estabelecimentos.dtypes

In [ ]:
# Verificando a proporção de valores nulos
df_estabelecimentos.isnull().sum()/df_estabelecimentos.shape[0]

#### Dados de CNAE

In [ ]:
df_cnae.head()

In [ ]:
df_cnae.dtypes

In [ ]:
# Verificando a proporção de valores nulos
df_cnae.isnull().sum()/df_cnae.shape[0]

#### Gerando a base final

Trazendo os dados dos estabelecimentos para a base da AMA utilizando o CNPJ como chave

In [ ]:
df_ama = pd.merge(df_ama, df_estabelecimentos, on='cnpj', how='left')

In [ ]:
df_ama.sample(5)

In [ ]:
df_estabelecimentos.dtypes

Trazendo os setores dos estabelecimentos para a base da AMA utiliando a CNAE como chave

In [ ]:
df_ama = pd.merge(df_ama, df_cnae, left_on='cnae_principal', right_on='codigo', how='left')

In [ ]:
df_ama.sample(5)

In [ ]:
# cnae_principal = codigo
df_ama.drop(columns='codigo', inplace=True)

### Salvando os arquivos em disco e desalocando o espaço em memória

#### Salvando os arquivos em parquet

Primeiro, instalei a biblioteca pyarrow para que possa utilizar read_parquet e to_parquet com o Pandas:

In [ ]:
!pip install pyarrow

Gravando a base principal no diretório corrente:

In [ ]:
df_ama.to_parquet('base_completa.parquet')

Gravando a base principal no diretório corrente (versão csv):

In [ ]:
df_ama.to_csv('base_completa.csv', index=False)

Gravando a base da ama no diretório corrente:

In [ ]:
df_estabelecimentos.to_parquet('estabelecimentos.parquet')

Gravando a base CNAE no diretório corrente:

In [ ]:
df_cnae.to_parquet('cnae.parquet')

#### Desalocando espaço em memória

In [ ]:
# Removando todos os DataFrames da memória
del df_ama; del df_estabelecimentos; del df_cnae

## Análise dos dados após gerar a base final

### Carga das bibliotecas

In [ ]:
# Carga das bibliotecas
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns

### Carga dos dados

In [ ]:
df = pd.read_csv('base_completa.csv')
df.sample(5)

### Tratamento dos dados

#### Informações gerais, conversões de tipos e ordenação dos dados pelo tempo

In [ ]:
# Dando uma olhada nas informações gerais da base
df.info()

In [ ]:
# CNAE não possui relevância neste momento, uma vez que já obtivemos a descrição dos CNPJs
df.drop(columns='cnae_principal', inplace=True)

In [ ]:
# Convertendo os tipos de algumas variáveis
df.data_emissao = pd.to_datetime(df.data_emissao)
df.situacao_cadastral = df.situacao_cadastral.astype(str)
df.id_matriz = df.id_matriz.astype(str)
df.dtypes

In [ ]:
# Ordenando os dados pelo tempo
df = df.sort_values(by='data_emissao')

#### Valores nulos

In [ ]:
# Verificando a proporção de valores ausentes
numero_linhas = df.shape[0]
proporcao_ausentes = (df.isnull().sum()/numero_linhas)*100
proporcao_ausentes

* Note que a variável nome_fantasia tem mais de 50% de seus valores ausentes. Sendo assim, não há muito o que se fazer com esta variável. Além disso, a variável emitente traz basicamente a mesma informação, porém sem dados ausentes.

* A variável bairro possui cerca de 0,002% dos bairros ausentes. Neste caso optaremos por remover as observações com os dados do bairro ausentes.

In [ ]:
# Removendo a variável nome_fantasia
df.drop(columns='nome_fantasia', inplace=True)

# Removendo as linhas cuja variável bairro está ausente
df.dropna(subset='bairro', inplace=True)

# Consultando novamente a proporção de valores ausentes
numero_linhas = df.shape[0]
proporcao_ausentes = (df.isnull().sum()/numero_linhas)*100
proporcao_ausentes

In [ ]:
# Verificando o novo formato da base
df.shape

#### Observando estatísticas descritivas da base

**Variáveis numéricas**

In [ ]:
# Estatísticas descritivas das variáveis numéricas
numericas = ['valor_nf', 'creditos']
df[numericas].describe()

Observando a distribuição dos valores de nota fiscal com o auxílio de um boxplot

In [ ]:
sns.boxplot(data=df, x='valor_nf')

Apesar de existir alguns outliers superiores, não será necessário desconsiderá-los. Será que há muitas ocorrências de valor_nf = 0?

In [ ]:
# Verificando o número de observações com valor de nf nulo
print(df[df['valor_nf'] == 0].shape[0])

# Será que algum desses valores retornam créditos?
print(df[df['valor_nf'] == 0]['creditos'].unique())

In [ ]:
# Removendo estas observações da base
df = df[df['valor_nf'] != 0]

**Variáveis categóricas**

In [ ]:
# Estatísticas descritivas das variáveis categóricas
display(df.loc[:, ~df.columns.isin(numericas + ['data_emissao'])].describe())

In [ ]:
df.loc[:, ~df.columns.isin(numericas + ['data_emissao'])].describe().loc['top', 'descricao']

* O número de valores únicos de CNPJs é maior que o de emitentes: investigar esta relação;
* O emitente que obteve mais registros vinculados a AMA foi a da TODA TORTA COMERCIO DE DOCES LTDA;
* Há 3 situações de créditos possíveis, onde a mais registrada é a de "Liberado";
* id_matriz registra duas categorias possíveis, o que pode ajudar na investigação da primeira conclusão;
* Há 5 situações cadastrais possíveis: será que em todas estas é possível retornar créditos?
* O bairro com o maior número de notas fiscais com CPFs vinculados a AMA foi o bairro do Cambuci;
* O setor de "Comércio varejista de mercadorias em geral, com predominância de produtos alimentícios" foi o setor com o maior número de compras registradas em prol da AMA.

#### Gerando novas features

Afim de fazermos análises temporais, iremos criar novas variáveis para identificar os meses/trimestres/semestres/anos em função da data de emissão:

In [ ]:
# Gerando identificadores numéricos para os meses
df['mes'] = df['data_emissao'].apply(lambda data: data.month)

In [ ]:
# Função responsável por gerar identificadores numéricos para os trimestre/semestres
def periodo_ano(mes):
    if 1 <= mes <= 3:
        trimestre = 1
        semestre = 1
    elif 4 <= mes <= 6:
        trimestre = 2
        semestre = 1
    elif 7 <= mes <= 9:
        trimestre = 3
        semestre = 2
    elif 10 <= mes <= 12:
        trimestre = 4
        semestre = 3
    else:
        trimestre = semestre = None
    return (trimestre, semestre)

In [ ]:
# Gerando identificadores numéricos para os trimestres
df['trimestre'] = df['mes'].apply(lambda data: periodo_ano(data)[0])

# Gerando identificadores numéricos para os semestres
df['semestre'] = df['mes'].apply(lambda data: periodo_ano(data)[1])

# Gerando identificadores numéricos para os anos
df['ano'] = df['data_emissao'].apply(lambda data: data.year)

In [ ]:
# Gerando identificadores de trimestre-semestre-ano
df['trimestre_semestre_ano'] = (df['trimestre'].astype(str) 
                                + '-'  + df['semestre'].astype(str) 
                                + '-' + df['ano'].astype(str))

In [ ]:
# Observando as variáveis criadas
df.head()

In [ ]:
# Observando os dtypes
df.dtypes

In [ ]:
# Criando labels para os meses
labels_meses = {1: "Jan",
                2: "Fev",
                3: "Mar",
                4: "Apr",
                5: "Mai",
                6: "Jun",
                7: "Jul",
                8: "Ago",
                9: "Sep",
                10: "Out",
                11: "Nov",
                12: "Dez"}
                
df['mes_label'] = df['mes'].apply(lambda numero_mes: labels_meses[numero_mes])

In [ ]:
# Observando as novas variáveis
df.head()

Também é interessante termos uma variável responsável por relacionar os créditos retornados com o valor da nota fiscal:

In [ ]:
df['retorno_percentual'] = (df['creditos']/df['valor_nf'])*100

In [ ]:
# Observando as variáveis criadas
df.head()

### Explorando as variáveis

#### relação entre cnpj e emitente

In [ ]:
# Deixando apenas as observações únicas da combinação entre cnpj e emitente
df_cnpj_emitente_unico = df.drop_duplicates(subset=['cnpj', 'emitente'])

# Agrupando por emitente e verificando o número de associados a cada cnpj
df_cnpj_emitente_unico.groupby('emitente')['cnpj'].count().sort_values(ascending=False)[:11]

Tomando a drogasil para estudo:

In [ ]:
df_cnpj_emitente_unico[df_cnpj_emitente_unico['emitente'] == 'RAIA DROGASIL S A'].head()

Observe que para um mesmo nome de emitente pode haver vários CNPJs, cada um destes representando uma loja distinta (verificado pelos bairros distintos, cada uma representando uma filial, pois id_matriz = 2).

#### situacao_credito

In [ ]:
df['situacao_credito'].value_counts()

In [ ]:
# Note que situações de crédito
df.loc[df['situacao_credito'] != 'Liberado', 'creditos'].unique()

Apesar de existirem valores de créditos diferentes de zero para as outras categorias diferentes de "Liberado", nesta análise iremos considerar apenas aquelas observações cujos créditos já estão com o status de liberado:

In [ ]:
df = df[df['situacao_credito'] == 'Liberado'] 
df.shape

#### situacao_cadastral

In [ ]:
# Observando a frequência de cada categoria
df.situacao_cadastral.value_counts()

In [ ]:
# Será que todas as situações cadastrais conseguem retornar créditos diferentes de 0?
df[df.creditos != 0].groupby('situacao_cadastral')['creditos'].count()

#### bairro

In [ ]:
# Verificando os 10 bairros que mais registraram compras vinculadas a AMA
df['bairro'].value_counts(ascending=False)[:20]

In [ ]:
# "Titularizando" o nome dos bairros
df['bairro'] = df['bairro'].str.title()
df['bairro'].head()

#### descricao

In [ ]:
# Verificando os 10 setores que mais registraram compras vinculadas a AMA
df['descricao'].value_counts(ascending=False)[:10]

Podemos dar uma melhorada no nome das categorias:

In [ ]:
# Quebrando as categorias por ' - '
df['descricao'] = df['descricao'].str.split(' - ')

In [ ]:
# Selecionando só a parte de interesse 
df['descricao'] = df['descricao'].apply(lambda palavra: palavra[1] if len(palavra) == 2 else palavra[0])

In [ ]:
# Observando 20 categorias aleatórias
df['descricao'].sample(20)

In [ ]:
# Mapeando as letras com acentos
gregas_acentuadas = {'Θ': 'é', 'Ω': 'ê', 'ß': 'á',
                     'Γ': 'â', 'φ': 'í', 'τ': 'ç',
                     '⌡': 'õ', 'π': 'ã', '≤': 'ó'}

# Corrigindo as letras regras
for grega, acentuada in gregas_acentuadas.items():
    df['descricao'] = df['descricao'].str.replace(grega, acentuada)

# Verificando se deu certo
df['descricao'].sample(20)

In [ ]:
# Removendo o trecho "Comércio varejista de"
df['descricao'] = df['descricao'].str.replace('Comércio varejista de ', '')
df['descricao'].sample(10)

In [ ]:
# Colocando a primeira letra da categoria como maiúscula
df['descricao'] = df['descricao'].str.capitalize()
df['descricao'].sample(10)

#### creditos e retorno percentual

Verificando a boxplot da variável retorno_percentual:

In [ ]:
sns.boxplot(data=df, y='retorno_percentual')

In [ ]:
# Visualizando as linhas com outliers acima de 80000%
display(df[df['retorno_percentual'] > 80000])

# Filtrando o df
df = df[df['retorno_percentual'] < 80000]

Como crédito é a principal variável de interesse, irei separar esta análise de acordo com sua relação com as demais variáveis.

In [ ]:
df.dtypes

**1) Mostre a evolução temporal dos créditos de acordo com o período de tempo da base**

In [ ]:
df1 = df[["ano","creditos"]].groupby("ano", as_index=False).agg("sum").sort_values('ano', ascending=False)

In [ ]:
janela, figura = plt.subplots(1, 1, figsize=(10, 6))

sns.barplot(ax=figura, data=df1, x='ano', y='creditos', palette='viridis')

# Configurações
figura.set_yticklabels(df1['creditos'], fontsize=15)
figura.set_ylabel('Créditos ao longo do ano', fontsize=15)

min_x = round(df1['ano'].min()*0.8)
max_x = round(df1['ano'].max()*1.05)
delta_x = round((max_x - min_x)/8)
figura.set_xlim(min_x, max_x)
figura.set_xticklabels(list(range(min_x, max_x, delta_x)), fontsize=15)
figura.set_xlabel('Créditos gerados', fontsize=15)

# Configurações da janela
janela.suptitle('Créditos gerados para a AMA ao longo dos anos',
                fontsize=18, x=0.5, y=0.95, weight='medium')

plt.show()
plt.savefig('1.jpeg')

**2) Mostre quais foram os 10 setores que mais contribuíram com a AMA**

Considerando todo o período de tempo da base, temos:

In [ ]:
df2 = df.groupby(by='descricao', as_index=False)['creditos'].sum().sort_values('creditos', ascending=False)[:10]
df2

In [ ]:
# Gerando a região do plot
janela, grafico = plt.subplots(1, 1, figsize=(12, 10))

# Realizando o plot de barras na região de gráfico criada
sns.barplot(data=df2, x='creditos', y='descricao', orient='h', palette='viridis', ax=grafico)

# Configurações
grafico.set_yticklabels(df2['descricao'], fontsize=15)
grafico.set_ylabel('Setor', fontsize=15)

min_x = round(df2['creditos'].min()*0.8)
max_x = round(df2['creditos'].max()*1.05)
delta_x = round((max_x - min_x)/8)
grafico.set_xlim(min_x, max_x)
grafico.set_xticklabels(list(range(min_x, max_x, delta_x)), fontsize=15)
grafico.set_xlabel('Créditos gerados', fontsize=15)

# Configurações da janela
janela.suptitle('Setores que mais geraram créditos para a AMA entre 2017 e 2022',
                fontsize=18, x=0.2, y=0.95, weight='medium')

plt.show()

**3) Mostre quais foram os 10 bairros mais contribuíram com a AMA**

In [ ]:
df3 = df.groupby('bairro', as_index=False)['creditos'].sum().sort_values('creditos', ascending=False)[:10]
df3

In [ ]:
# Gerando a região do plot
janela, grafico = plt.subplots(1, 1, figsize=(12, 10))

# Realizando o plot de barras na região de gráfico criada
sns.barplot(data=df3, x='creditos', y='bairro', orient='h', palette='viridis', ax=grafico)

# Configurações
grafico.set_yticklabels(df3['bairro'], fontsize=15)
grafico.set_ylabel('Bairros', fontsize=15)

min_x = round(df3['creditos'].min()*0.8)
max_x = round(df3['creditos'].max()*1.05) + 100
delta_x = round((max_x - min_x)/8)
grafico.set_xlim(min_x, max_x)
grafico.set_xticklabels(list(range(min_x, max_x, delta_x)), fontsize=15)
grafico.set_xlabel('Créditos gerados', fontsize=15)

# Configurações da janela
janela.suptitle('Bairros que mais geraram créditos para a AMA entre 2017 e 2022',
                fontsize=18, x=0.5, y=0.95)

plt.show()

In [ ]:
df['descricao'].nunique()

**4) Mostre quais foram as 10 compras que tiveram um maior retorno percentual - PODE SER SÓ A TABELA**

In [ ]:
selecionadas_4 = ['emitente', 'bairro', 'valor_nf', 'creditos', 'retorno_percentual']
df4 = df.sort_values('retorno_percentual', ascending=False)[selecionadas_4][:10]
df4

In [ ]:
sns.boxplot(x=df['retorno_percentual'])

In [ ]:
# Gerando a região do plot
janela, grafico = plt.subplots(2, 1, figsize=(15, 20))

## Primeiro plot: Gráfico de barras relacionando o valor da compra com o retorno percentual
# Realizando o plot de barras na região de gráfico criada
sns.barplot(data=df4, x='creditos', y='retorno_percentual', orient='v', palette='viridis', ax=grafico[1])

# # Configurações
# grafico[1].set_yticks(df4['retorno_percentual'])
# grafico[1].set_yticklabels(df4['retorno_percentual'].values, fontsize=15)
# grafico[1].set_ylabel('Retorno percentual', fontsize=15)

# min_x = round(df4['creditos'].min()*0.8)
# max_x = round(df4['creditos'].max()*1.05)
# print(grafico[1].set_xlim())
# grafico[1].set_xlim(min_x, max_x)
# grafico[1].set_xticklabels(list(range(min_x, max_x, delta_x)), fontsize=15)
# grafico[1].set_xlabel('Créditos gerados', fontsize=15)

# # Configurações da janela
# janela.suptitle('saída',
#                 fontsize=18, x=0.5, y=0.95)

plt.show()

**5) Quais são os setores que mais contribuiram com a AMA ao longo do tempo total?**

In [ ]:
# df_sep_a= filtered_df.groupby([pd.Grouper(key='Ano')]).agg({'Créditos': 'sum'})
# df_sep_a=df_sep_a.reset_index(drop=False)
# fig = px.bar(df_sep_a, x="Ano", y="Créditos", color="Créditos", title="Soma de Créditos por ano",color_continuous_scale=px.colors.sequential.Viridis)
# fig.show()

In [ ]:
# segmento_full = df_completo.groupby("descricao").agg({"valor_nf": lambda x:x.mean(),
#                                                       "creditos": lambda x:x.mean(),
#                                                       "id_matriz": lambda x:x.mean(),
#                                                       "emitente": lambda x:x.mode(),
#                                                       "situacao_credito": lambda x:x.mode(),
#                                                       "bairro": lambda x:x.mode()}).sort_values(by="creditos", ascending=False).iloc[:10]

In [ ]:
# fig = plt.figure()
# bar = ''
# def buildmebarchart(i=int):
#     iv = min(i, len(df1.index)-1) #the loop iterates an extra one time, which causes the dataframes to go out of bounds. This was the easiest (most lazy) way to solve this :)
#     objects = df1.max().index
#     y_pos = np.arange(len(objects))
#     performance = df1.iloc[[iv]].values.tolist()[0]
#     if bar == 'vertical':
#         plt.bar(y_pos, performance, align='center', color=['red', 'green', 'blue', 'orange'])
#         plt.xticks(y_pos, objects)
#         plt.ylabel('Deaths')
#         plt.xlabel('Countries')
#         plt.title('Deaths per Country \n' + str(df1.index[iv].strftime('%y-%m-%d')))
#     else:
#         plt.barh(y_pos, performance, align='center', color=['red', 'green', 'blue', 'orange'])
#         plt.yticks(y_pos, objects)
#         plt.xlabel('Deaths')
#         plt.ylabel('Countries')
# animator = ani.FuncAnimation(fig, buildmebarchart, interval=100)
# plt.show()

In [ ]:
df_5 = df.groupby(['descricao', 'trimestre_semestre_ano'], as_index=False)['creditos'].sum()

In [ ]:
df_5.head()

In [ ]:
params = {'legend.fontsize': '20',
          'figure.figsize': (80, 30),
         'axes.labelsize': '15',
         'axes.titlesize':'x-large',
         'xtick.labelsize':'35',
         'ytick.labelsize':'35'}
plt.rcParams.update(params)

# Gerando a estrutura com matplotlib
janela, graficos = plt.subplots(3, 2)
janela.subplots_adjust(hspace=None)

# Criando uma nova variável
df_5['trimestre'] = df_5['trimestre_semestre_ano'].str.split('-').apply(lambda linha: linha[0]).astype(int)
df_5['ano'] = df_5['trimestre_semestre_ano'].str.split('-').apply(lambda linha: linha[2]).astype(int)

# Pegando uma seção da base
recorte_df5 = df_5[df_5['creditos'] != 0]

# Fazendo o plot 
sns.barplot(x="creditos", y="descricao", data=recorte_df5[recorte_df5['ano'] == 2017][:10],
            hue='trimestre', orient='h', ax=graficos[0, 0], )
    

# Configurações
graficos[0, 0].set_xlabel('Eixo x')
graficos[0, 0].set_ylabel('Eixo y')
graficos[0, 0].set_title('Título')
janela.suptitle('Título da janela', fontsize='20')

# Fazendo o plot 
sns.barplot(x="creditos", y="descricao", data=recorte_df5[recorte_df5['ano'] == 2018][:10],
            hue='trimestre', orient='h', ax=graficos[0, 1])
    

# Configurações
graficos[0, 1].set_xlabel('Eixo x')
graficos[0, 1].set_ylabel('Eixo y')
graficos[0, 1].set_title('Título')

# Fazendo o plot 
sns.barplot(x="creditos", y="descricao", data=recorte_df5[recorte_df5['ano'] == 2019][:10],
            hue='trimestre', orient='h', ax=graficos[1, 0])
    

# Configurações
graficos[1, 0].set_xlabel('Eixo x')
graficos[1, 0].set_ylabel('Eixo y')
graficos[1, 0].set_title('Título')

# Fazendo o plot 
sns.barplot(x="creditos", y="descricao", data=recorte_df5[recorte_df5['ano'] == 2020][:10],
            hue='trimestre', orient='h', ax=graficos[1, 1])
    

# Configurações
graficos[1, 1].set_xlabel('Eixo x')
graficos[1, 1].set_ylabel('Eixo y')
graficos[1, 1].set_title('Título')

# Fazendo o plot 
sns.barplot(x="creditos", y="descricao", data=recorte_df5[recorte_df5['ano'] == 2021][:10],
            hue='trimestre', orient='h', ax=graficos[2, 0])
    

# Configurações
graficos[2, 0].set_xlabel('Eixo x')
#graficos[2, 0].set_xticklabels(size='12')
graficos[2, 0].set_ylabel('Eixo y')
graficos[2, 0].set_title('Título')
janela.suptitle('Título da janela', fontsize='20')

plt.tight_layout()
janela.savefig('saida.jpeg')

In [ ]:
df_5['trimestre'] = df_5['trimestre_semestre_ano'].str.split('-').apply(lambda linha: linha[0])

In [ ]:
df['data_emissao'].value_counts(ascending=False)

In [ ]:
recorte_df5 = df_5[df_5['creditos'] != 0]

sns.barplot(x="creditos", y="descricao", data=recorte_df5[:20], hue='trimestre', orient='h')

In [ ]:
# filtered_df.groupby([pd.Grouper(key='Data Registro', freq='m')]).agg({'Créditos': 'sum'})


df_5 = df.groupby([pd.Grouper(key='data_emissao', freq='m')]).agg({'creditos': 'sum'})
df_5[:20]

df_5 = df_5.groupby(['descricacao',''], as_index=False)['quantity'].sum()

In [ ]:
df = df.groupby(['id','product'], as_index=False)['quantity'].sum()

**7) Quais são os setores que não geram créditos?**

In [ ]:
df_7 = df.groupby('descricao')['creditos'].sum()[df.groupby('descricao')['creditos'].sum() == 0]
df_7

**8) Quais foram as 10 lojas que mais contribuiram com a AMA? Faça uma análise ao longo de todo o tempo, e em seguida faça a mesma análise por ano.**

In [ ]:
df_8 = df.groupby('emitente')['creditos'].sum().sort_values(ascending=False)
df_8.head()

**9) O valor da compra influencia a pessoa a registrar o CPF na hora da compra?**

In [ ]:
df_9 = df[df['descricao'] == 'ComΘrcio varejista de artigos de ≤ptica']
df_9[df['creditos'] == 0]

**10) Calcule o quanto poderia ter sido revertido em dinheiro para a AMA com as compras que não tiveram o CPF vinculado (NÃO SEI SE É POSSÍVEL DETERMINAR. INVESTIGAR...).**

In [ ]:
#0c185c
#0065ab
#00b3d2
#4affd8

**11) Quais os meses onde a arrecadação foi maior?** 